In [ ]:
suppressPackageStartupMessages({
    library(DropletUtils)
    library(SingleCellExperiment)
    library(scuttle)
    library(Seurat)
    library(SeuratWrappers)
    library(stringr)
    library(dplyr)
    library(data.table)
    library(Matrix)
    library(patchwork)
    library(ggplot2)
    library(sceasy)
})

options(repr.plot.width = 16, repr.plot.height = 8)

In [ ]:
Atlas <- readRDS('Atlas.Rds')

In [ ]:
Idents(Atlas) <- Atlas@meta.data$CellType

In [ ]:
head(Idents(Atlas))

In [ ]:
mesenchymal_subset <- subset(Atlas, idents=c('Periosteal progenitors','Periosteum','Endosteum','Resting zone chondrocytes','Proliferative chondrocytes','Pre-hypertrophic chondrocytes','Hypertrophic chondrocytes','Chondrocytes','Chondroprogenitors','Osteoprogenitors','Pre-osteoblasts','Osteoblasts','Osteocytes','Vascular endothelial cells','Lymphatic endothelial cells','Proximal limb bud mesenchyme','Intermediate limb bud mesenchyme','Distal limb bud mesenchyme','Myogenic stem cells','Muscle progenitors','Skeletal muscle cells'))

In [ ]:
Idents(mesenchymal_subset) <- as.factor(mesenchymal_subset$DevTP)

In [ ]:
head(Idents(mesenchymal_subset))

In [ ]:
dev_subset <- subset(mesenchymal_subset, idents=c('P13','P6','E18.5','E15.5','E13.5','E12.5','E11.5','E10.5','P21'))

In [ ]:
dev_subset$DevTP <- droplevels.factor(dev_subset$DevTP)

In [ ]:
dev_subset@reductions$UMAP <- dev_subset@reductions$mde_scanvi

In [ ]:
plot_cells(cds, label_groups_by_cluster=FALSE,  color_cells_by = "CellType", reduction_method = 'UMAP')

In [ ]:
dev_subset

In [ ]:
cds <- as.cell_data_set(GP)


In [ ]:
cds <- cluster_cells(cds)


In [ ]:
cds <- learn_graph(cds)


In [ ]:
plot_cells(cds, color_cells_by = "CellType",label_groups_by_cluster=FALSE,label_leaves=FALSE, label_branch_points=FALSE)


In [ ]:
cds <- order_cells(cds)


In [ ]:
dev_subset@meta.data$pseudotime <- pseudotime(cds)


In [ ]:
head(dev_subset@meta.data$pseudotime)

In [ ]:
sceasy::convertFormat(dev_subset, from="seurat", to="anndata",
                       outFile='KO.h5ad')

In [ ]:
dev_subset

In [ ]:
options(repr.plot.width = 17, repr.plot.height = 10)
p1 <- DimPlot(dev_subset, group.by='NandC',reduction = "mde_scanvi",order = c('39 - Keratinocytes', '38 - Neurons', '37 - Platelets', '36 - Erythroid cells', '35 - Immune cells', '34 - Epithelial cells', '33 - AER', '32 - Lymphatic endothelial cells', '31 - Vascular endothelial cells', '30 - Perivascular cells', '29 - Skeletal muscle cells', '28 - Muscle progenitors', '27 - Myogenic stem cells', '26 - Fibroblasts', '25 - Osteoclasts', '24 - Endosteum', '23 - Periosteum', '22 - Periosteal progenitors', '21 - Osteocytes', '20 - Fast proliferating osteoblasts', '19 - Osteoblasts', '18 - Pre-osteoblasts', '17 - Osteoprogenitors', '16 - Tenocytes', '15 - Tenocyte precursors', '14 - Joint precursors', '13 - Articular chondrocytes', '12 - Hypertrophic chondrocytes', '11 - Pre-hypertrophic chondrocytes', '10 - Proliferative chondrocytes', '9 - Resting zone chondrocytes', '8 - Chondrocytes', '7 - Chondroprogenitors', '6 - Fast proliferating cells', '5 - ZPA', '4 - Distal limb bud mesenchyme', '3 - Intermediate limb bud mesenchyme', '2 - Proximal limb bud mesenchyme', '1 - BMSCs')
, pt.size = 0.3, label = F , label.size=6 ,label.color = 'black', repel=TRUE, raster=FALSE, cols= c(`1 - BMSCs` = "#66C2A5",
`2 - Proximal limb bud mesenchyme` = "#F781BF",
`3 - Intermediate limb bud mesenchyme` = "#B3DE69",
`4 - Distal limb bud mesenchyme` = "#8DA0CB",
`5 - ZPA` = "#949086",
`6 - Fast proliferating cells` = "#FFCCCB",
`7 - Chondroprogenitors` = "#666666",
`8 - Chondrocytes` = "#B3B3B3",
`9 - Resting zone chondrocytes` = "#8DD3C7",
`10 - Proliferative chondrocytes` = "#D4A6CD",
`11 - Pre-hypertrophic chondrocytes` = "#FB8072",
`12 - Hypertrophic chondrocytes` = "#FFD700",
`13 - Articular chondrocytes` = "#0B0F5E",
`14 - Joint precursors` = "#377EB8",
`15 - Tenocyte precursors` = "#FDB462",
`16 - Tenocytes` = "#DD2400",
`17 - Osteoprogenitors` = "#FFFF33",
`18 - Pre-osteoblasts` = "#A65628",
`19 - Osteoblasts` = "#FF7F50",
`20 - Fast proliferating osteoblasts` = "#90EE90",
`21 - Osteocytes` = "#003e00",
`22 - Periosteal progenitors` = "#FFD92F",
`23 - Periosteum` = "#FF7F00",
`24 - Endosteum` = "#502c49",
`25 - Osteoclasts` = "#A6D854",
`26 - Fibroblasts` = "#B3B3B3",
`27 - Myogenic stem cells` = "#5E885e",
`28 - Muscle progenitors` = "#961900",
`29 - Skeletal muscle cells` = "#F845a3",
`30 - Perivascular cells` = "#8DA0CB",
`31 - Vascular endothelial cells` = "#7e18c6",
`32 - Lymphatic endothelial cells` = "#B0D30A",
`33 - AER` = "#640738",
`34 - Epithelial cells` = "#A2CFFE",
`35 - Immune cells` = "#66C2A5",
`36 - Erythroid cells` = "#D4A6CD",
`37 - Platelets` = "#B15928",
`38 - Neurons` = "#A0d4a4",
`39 - Keratinocytes` = "#8B4513"
)) +
        xlab("UMAP 1") + ylab("UMAP 2") +
        theme(axis.title = element_text(size = 15)) +
        theme(plot.title = element_blank()) +
        theme(axis.text.x = element_blank(),
              axis.ticks.x = element_blank(),
              axis.text.y = element_blank(),
              axis.ticks.y = element_blank())    

p1 

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
FeaturePlot_scCustom(seurat_object = dev_subset, reduction = 'mde_scanvi',colors_use = viridis_plasma_light_high, features = "pseudotime")